# Neo4j AI Agent Template - Complete Guide

## 🎯 What This Template Creates

This notebook creates an intelligent database assistant that lets you talk to your Neo4j database in **plain English** instead of writing complex database queries.

### Before vs After
- **Before**: You'd need to learn Cypher (Neo4j's query language) to ask your database questions
- **After**: You can ask "Show me all customers with overdue invoices" and the AI figures out the database query for you

### Real-World Example
Instead of writing:
```cypher
MATCH (c:Customer)-[:HAS_INVOICE]->(i:Invoice)
WHERE i.dueDate < date() AND i.status = 'unpaid'
RETURN c.name, i.amount
```
You just ask: **"Which customers have unpaid invoices that are past due?"**

### Technology Stack
- **Neo4j**: Graph database (stores connected data like customers, invoices, subscriptions)
- **Google Vertex AI**: Google's AI service that understands natural language
- **Google ADK**: Agent Development Kit for building intelligent agents
- **MCP Toolbox**: Model Context Protocol for database interactions
- **Gemini**: Google's AI model that converts questions into database queries

### What You'll Have When Done
A deployed AI assistant that your team can use to get insights from your data without needing to know database programming.

---

## 📋 Prerequisites
1. **Google Cloud Project** with billing enabled
2. **Neo4j Database** (Neo4j Aura or self-hosted)
3. **Basic understanding** of your data structure
4. **5-15 minutes** for deployment

---

## 🗂️ Template Structure

### Phase 1: Environment Setup
- Install required packages
- Authenticate with Google Cloud
- Configure your settings

### Phase 2: Local Development & Testing
- Create the AI agent
- Test locally with your database
- Verify functionality

### Phase 3: Deployment Preparation
- Prepare deployment configuration
- Set up environment variables
- Create deployable agent class

### Phase 4: Cloud Deployment
- Deploy to Vertex AI Agent Engine
- Test the deployed agent
- Verify cloud functionality

Let's begin! 🚀

# 🔧 Phase 1: Environment Setup

In this phase, we'll install all the necessary tools and set up authentication.

## Step 1.1: Install Required Packages

**What this does**: Downloads and installs the software tools you need - think of it like installing apps on your phone.

**Packages installed**:
- `google-cloud-aiplatform[adk,agent_engines]`: Google's AI platform with Agent Development Kit
- `neo4j`: Official Neo4j Python driver for database connections
- `requests`: HTTP library for web requests

**Time**: ~30-60 seconds

In [1]:
print("🔄 Installing required packages...")
print("   This may take 30-60 seconds...")

!pip install --upgrade --quiet "google-cloud-aiplatform[adk,agent_engines]>=1.56.0" neo4j>=5.15.0 requests==2.32.3

print("✅ Packages installed successfully!")
print("   - Google Cloud AI Platform with ADK")
print("   - Neo4j Python Driver")
print("   - Requests library")

🔄 Installing required packages...
   This may take 30-60 seconds...
✅ Packages installed successfully!
   - Google Cloud AI Platform with ADK
   - Neo4j Python Driver
   - Requests library


## Step 1.2: Restart Runtime

**What this does**: Restarts your coding environment so it can use the newly installed tools. Like restarting your computer after installing new software.

**Note**: After running this cell, the notebook will restart. This is normal and expected.

In [2]:
print("🔄 Restarting runtime to load new packages...")
print("   The notebook will restart automatically.")
print("   Continue with the next cell after restart.")

# Restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

🔄 Restarting runtime to load new packages...
   The notebook will restart automatically.
   Continue with the next cell after restart.


{'status': 'ok', 'restart': True}

## Step 1.3: Authenticate with Google Cloud

**What this does**: Logs you into Google Cloud so you can use their AI services.

**When to run**: Only needed if you're using Google Colab. Skip if running locally with gcloud auth.

**What happens**: A popup will appear asking you to authorize access to your Google Cloud account.

In [3]:
print("🔐 Authenticating with Google Cloud...")

try:
    from google.colab import auth
    auth.authenticate_user()
    print("✅ User authenticated successfully!")
    print("   You can now access Google Cloud services.")
except ImportError:
    print("ℹ️  Not running in Colab - assuming local authentication is set up.")
    print("   Make sure you've run 'gcloud auth login' locally.")

🔐 Authenticating with Google Cloud...
✅ User authenticated successfully!
   You can now access Google Cloud services.


## Step 1.4: Configure Your Settings

**What this does**: Sets up your "address book" - tells the code where to find your Google Cloud project, Neo4j database, and login credentials.

**🚨 IMPORTANT**: Replace the placeholder values below with your actual credentials:

### Google Cloud Settings
- `PROJECT_ID`: Your Google Cloud project ID
- `LOCATION`: Your preferred region (e.g., 'us-central1', 'europe-west4')
- `STAGING_BUCKET`: Your GCS bucket for storing deployment files

### Neo4j Settings
- `NEO4J_URI`: Your Neo4j connection URI
- `NEO4J_USERNAME`: Your Neo4j username (usually 'neo4j')
- `NEO4J_PASSWORD`: Your Neo4j password
- `NEO4J_DATABASE`: Your database name (usually 'neo4j')

### Agent Settings
- `MODEL_NAME`: The AI model to use (recommend 'gemini-2.5-flash')
- `AGENT_DISPLAY_NAME`: A unique name for your agent

In [7]:
print("⚙️  Interactive Configuration Setup...")
print("   Enter your credentials when prompted (press Enter for defaults where shown)")

# =============================================================================
# 📝 INTERACTIVE CONFIGURATION - Enter your actual credentials
# =============================================================================

print("\n--- Google Cloud Configuration ---")
PROJECT_ID = input("🔴 Google Cloud Project ID: ").strip()
while not PROJECT_ID:
    print("   ❌ Project ID is required!")
    PROJECT_ID = input("🔴 Google Cloud Project ID: ").strip()

LOCATION = input(f"🌍 Region [europe-west4]: ").strip() or "europe-west4"

STAGING_BUCKET = input("📦 GCS Staging Bucket (gs://bucket-name): ").strip()
while not STAGING_BUCKET or not STAGING_BUCKET.startswith("gs://"):
    if STAGING_BUCKET and not STAGING_BUCKET.startswith("gs://"):
        print("   ❌ Bucket must start with 'gs://'")
        STAGING_BUCKET = input("📦 GCS Staging Bucket (gs://bucket-name): ").strip()
    else:
        print("   ❌ Staging bucket is required!")
        STAGING_BUCKET = input("📦 GCS Staging Bucket (gs://bucket-name): ").strip()

print("\n--- Neo4j Configuration ---")
NEO4J_URI = input("🗄️  Neo4j URI (neo4j+s://xxxxx.databases.neo4j.io): ").strip()
while not NEO4J_URI:
    print("   ❌ Neo4j URI is required!")
    NEO4J_URI = input("🗄️  Neo4j URI: ").strip()

NEO4J_USERNAME = input("👤 Neo4j Username [neo4j]: ").strip() or "neo4j"

NEO4J_PASSWORD = input("🔑 Neo4j Password: ").strip()
while not NEO4J_PASSWORD:
    print("   ❌ Neo4j password is required!")
    NEO4J_PASSWORD = input("🔑 Neo4j Password: ").strip()

NEO4J_DATABASE = input("🗃️  Neo4j Database [neo4j]: ").strip() or "neo4j"

print("\n--- Agent Configuration ---")
print("🧠 Available models:")
print("   - gemini-2.5-flash (fastest)")
print("   - gemini-2.5-pro (recommended: most capable)")

MODEL_NAME = input("🤖 Model [gemini-2.5-flash]: ").strip() or "gemini-2.5-pro"
while MODEL_NAME not in ["gemini-2.5-flash", "gemini-2.5-pro"]:
    print("   ❌ Please choose either 'gemini-2.5-flash' or 'gemini-2.5-pro'")
    MODEL_NAME = input("🤖 Model [gemini-2.5-pro]: ").strip() or "gemini-2.5-pro"

AGENT_DISPLAY_NAME = input("🏷️  Agent Display Name: ").strip()
while not AGENT_DISPLAY_NAME:
    print("   ❌ Agent name is required!")
    AGENT_DISPLAY_NAME = input("🏷️  Agent Display Name: ").strip()

# =============================================================================

print("\n🔍 Configuration validation:")
print("✅ PROJECT_ID: Set")
print("✅ STAGING_BUCKET: Valid format")
print("✅ NEO4J_URI: Set")
print("✅ NEO4J_PASSWORD: Set")
print("✅ AGENT_DISPLAY_NAME: Set")

print("\n✅ All configuration completed successfully!")
print(f"   📋 Project: {PROJECT_ID}")
print(f"   🌍 Region: {LOCATION}")
print(f"   🗄️  Database: {NEO4J_URI.split('@')[1] if '@' in NEO4J_URI else NEO4J_URI}")
print(f"   🤖 Agent: {AGENT_DISPLAY_NAME}")
print(f"   🧠 Model: {MODEL_NAME}")
print(f"   📦 Staging: {STAGING_BUCKET}")

print("\n🎯 Ready to proceed with agent creation!")

⚙️  Interactive Configuration Setup...
   Enter your credentials when prompted (press Enter for defaults where shown)

--- Google Cloud Configuration ---
🔴 Google Cloud Project ID: 881765721010 
🌍 Region [europe-west4]: europe-west4
📦 GCS Staging Bucket (gs://bucket-name): gs://vertex_ai_agent_engine_test_disco

--- Neo4j Configuration ---
🗄️  Neo4j URI (neo4j+s://xxxxx.databases.neo4j.io): neo4j+s://agentspace-c4x9-c86d.endpoints.neo4j.io
👤 Neo4j Username [neo4j]: neo4j
🔑 Neo4j Password: 8hZTnHo5gyXe1V4bSqlbjgVsyYjESB9Oh1clbXIi_0k
🗃️  Neo4j Database [neo4j]: neo4j

--- Agent Configuration ---
🧠 Available models:
   - gemini-2.5-flash (fastest)
   - gemini-2.5-pro (recommended: most capable)
🤖 Model [gemini-2.5-flash]: gemini-2.5-flash
🏷️  Agent Display Name: agent_neo4j_v5

🔍 Configuration validation:
✅ PROJECT_ID: Set
✅ STAGING_BUCKET: Valid format
✅ NEO4J_URI: Set
✅ NEO4J_PASSWORD: Set
✅ AGENT_DISPLAY_NAME: Set

✅ All configuration completed successfully!
   📋 Project: 881765721010


# 🧪 Phase 2: Local Development & Testing

In this phase, we'll create the AI agent and test it locally with your Neo4j database.

## Step 2.1: Import Required Libraries

**What this does**: Brings in the specific tools you need from Google's AI toolkit and Neo4j. Like opening the apps you need for a project.

**Libraries imported**:
- `vertexai`: Google's AI platform
- `google.adk.agents`: Agent Development Kit
- `neo4j`: Database driver
- `json`, `logging`: Utility libraries

In [8]:
print("📦 Importing required libraries...")

import os
import logging
import json
import vertexai
from google.adk.agents import Agent
from vertexai.preview import reasoning_engines
from vertexai import agent_engines
from neo4j import GraphDatabase
from datetime import date, datetime

print("✅ Libraries imported successfully!")
print("   - Vertex AI platform")
print("   - Google ADK agents")
print("   - Neo4j database driver")
print("   - Supporting utilities")

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

📦 Importing required libraries...
✅ Libraries imported successfully!
   - Vertex AI platform
   - Google ADK agents
   - Neo4j database driver
   - Supporting utilities


## Step 2.2: Initialize Vertex AI

**What this does**: Starts up Google's AI service using the settings you provided in Step 1.4.

**This step**:
- Connects to your Google Cloud project
- Sets up the AI platform in your specified region
- Prepares the staging bucket for deployments

In [9]:
print("🚀 Initializing Vertex AI...")

try:
    vertexai.init(project=PROJECT_ID, location=LOCATION, staging_bucket=STAGING_BUCKET)
    print("✅ Vertex AI initialized successfully!")
    print(f"   📋 Project: {PROJECT_ID}")
    print(f"   🌍 Region: {LOCATION}")
    print(f"   📦 Staging Bucket: {STAGING_BUCKET}")

    # Set environment variables for the agent functions
    os.environ["NEO4J_URI"] = NEO4J_URI
    os.environ["NEO4J_USERNAME"] = NEO4J_USERNAME
    os.environ["NEO4J_PASSWORD"] = NEO4J_PASSWORD
    os.environ["NEO4J_DATABASE"] = NEO4J_DATABASE

    print("✅ Environment variables set for Neo4j connection.")

except Exception as e:
    print(f"❌ ERROR: Failed to initialize Vertex AI: {e}")
    print("\n🔧 Troubleshooting:")
    print("   1. Verify your PROJECT_ID is correct")
    print("   2. Ensure the Vertex AI API is enabled in your project")
    print("   3. Check your staging bucket exists and is accessible")
    raise

🚀 Initializing Vertex AI...
✅ Vertex AI initialized successfully!
   📋 Project: 881765721010
   🌍 Region: europe-west4
   📦 Staging Bucket: gs://vertex_ai_agent_engine_test_disco
✅ Environment variables set for Neo4j connection.


## Step 2.3: Create Agent Tool Functions

**What this does**: Creates the core functions that your AI agent will use to interact with your Neo4j database.

**Functions created**:
1. **`_make_serializable`**: Helper function to handle Neo4j date objects
2. **`_execute_query`**: Private function to run Cypher queries safely
3. **`get_neo4j_schema`**: Gets database structure (nodes, relationships, properties)
4. **`execute_cypher_query`**: Executes read-only queries for data retrieval

**Security**: The functions are designed to be read-only to prevent accidental data modification.

In [10]:
print("🔧 Creating agent tool functions...")

def _make_serializable(obj):
    """
    Helper function that recursively finds non-serializable objects (like dates)
    and converts them to strings for JSON serialization.

    This fixes issues with Neo4j date/time objects that can't be directly
    converted to JSON.
    """
    if isinstance(obj, dict):
        return {k: _make_serializable(v) for k, v in obj.items()}
    if isinstance(obj, list):
        return [_make_serializable(i) for i in obj]
    # Check for Neo4j and standard Python date/datetime objects
    if isinstance(obj, (date, datetime)) or type(obj).__module__.startswith('neo4j.time'):
        return obj.isoformat()
    return obj

def _execute_query(query: str, params: dict) -> str:
    """
    Private helper function to execute a Cypher query safely.

    Creates a new database connection for each query to ensure
    statelessness for cloud deployment.

    Args:
        query: The Cypher query to execute
        params: Parameters for the query

    Returns:
        JSON string with query results or error information
    """
    try:
        # Create a new driver connection for each query
        with GraphDatabase.driver(
            os.environ["NEO4J_URI"],
            auth=(os.environ["NEO4J_USERNAME"], os.environ["NEO4J_PASSWORD"]),
            database=os.environ["NEO4J_DATABASE"]
        ) as driver:
            with driver.session() as session:
                result = session.run(query, params)
                records = [record.data() for record in result]
                # Convert records to be JSON-serializable
                serializable_records = _make_serializable(records)
                return json.dumps(serializable_records)
    except Exception as e:
        logging.error(f"Neo4j query failed: {e}")
        return json.dumps({"error": str(e)})

def get_neo4j_schema() -> str:
    """
    Retrieves the database schema, including all node labels, their properties,
    and relationship types.

    This is the most important tool for the agent to understand the database structure.
    The agent should call this first before attempting any queries.

    Returns:
        JSON string containing nodes, relationships, and constraints
    """
    logging.info("Executing tool: get_neo4j_schema")
    # Use db.schema.visualization() to get comprehensive schema information
    statement = "CALL db.schema.visualization()"
    return _execute_query(statement, {})

def execute_cypher_query(cypher_query: str) -> str:
    """
    Executes a read-only Cypher query against the database.

    Use this tool after you have a good understanding of the database schema
    from the get_neo4j_schema tool.

    Args:
        cypher_query: The Cypher query to execute (read-only only)

    Returns:
        JSON string with query results or error information

    Security:
        Only allows read operations (MATCH, RETURN). Blocks write operations
        (CREATE, MERGE, SET, DELETE) for safety.
    """
    logging.info(f"Executing tool: execute_cypher_query with query='{cypher_query}'")

    # Security check: Only allow read-only queries
    write_keywords = ["CREATE", "MERGE", "SET", "DELETE", "REMOVE"]
    if any(keyword in cypher_query.upper() for keyword in write_keywords):
        return json.dumps({
            "error": "This tool can only execute read-only queries (MATCH, RETURN). Write operations are not allowed for safety."
        })

    return _execute_query(cypher_query, {})

print("✅ Agent tool functions created successfully!")
print("   🔍 get_neo4j_schema - Discovers database structure")
print("   📊 execute_cypher_query - Runs read-only queries")
print("   🔒 Security: Write operations are blocked")
print("   🔄 JSON serialization: Handles Neo4j date objects")

🔧 Creating agent tool functions...
✅ Agent tool functions created successfully!
   🔍 get_neo4j_schema - Discovers database structure
   📊 execute_cypher_query - Runs read-only queries
   🔒 Security: Write operations are blocked
   🔄 JSON serialization: Handles Neo4j date objects


## Step 2.4: Create the AI Agent

**What this does**: Creates your AI assistant using Google's Agent Development Kit (ADK). This agent combines Google's language model with your database tools.

**Agent capabilities**:
- Understands natural language questions
- Automatically discovers your database structure
- Converts questions to Cypher queries
- Returns results in human-readable format

**How it works**:
1. Agent receives your question
2. First calls `get_neo4j_schema` to understand your data
3. Constructs appropriate Cypher query
4. Executes query using `execute_cypher_query`
5. Formats results for you

In [11]:
print("🤖 Creating AI agent with Google ADK...")

# Create the agent using Google's Agent Development Kit
neo4j_agent = Agent(
    name="neo4j_database_agent",
    model=MODEL_NAME,
    description="An intelligent agent that can inspect and query any Neo4j database using natural language.",
    instruction=(
    "You are an expert Neo4j database assistant. Your role is to help users get insights "
    "from their Neo4j database using natural language queries.\n\n"
    "WORKFLOW:\n"
    "1. ALWAYS start by calling `get_neo4j_schema` to understand the database structure\n"
    "2. Analyze the schema to understand nodes, relationships, and properties\n"
    "3. Use `execute_cypher_query` to construct and run appropriate Cypher queries\n"
    "4. Present results in a clear, human-readable format\n\n"
    "BEST PRACTICES:\n"
    "- Use exact node labels and relationship types from the schema\n"
    "- Be intelligent about property matching - try common variations (mrr/MRR, name/productName, etc.)\n"
    "- Add LIMIT clauses for queries that might return many results (default: 10) but NOT for aggregations\n"
    "- Handle errors gracefully and provide helpful error messages\n"
    "- Format results as tables or lists when appropriate\n"
    "- Make reasonable assumptions about data relationships before asking for clarification\n"
    "- If a property seems related to what the user wants, try it first\n\n"
    "QUERY CONSTRUCTION:\n"
    "- For financial queries, look for properties like: amount, mrr, arr, value, cost, price\n"
    "- For product queries, look for: name, productName, serviceName, description\n"
    "- For date queries, look for: date, createdDate, startDate, endDate, issueDate\n"
    "- Try the most likely property combinations before asking for clarification\n\n"
    "SECURITY:\n"
    "- Only use read-only operations (MATCH, RETURN)\n"
    "- Never modify data (no CREATE, MERGE, SET, DELETE)\n\n"
    "Always provide comprehensive, accurate responses. Be confident and make intelligent assumptions based on the discovered schema."
    ),
    tools=[
        get_neo4j_schema,      # Tool to discover database structure
        execute_cypher_query,  # Tool to run read-only queries
    ],
)

print("✅ AI agent created successfully!")
print(f"   🤖 Agent Name: {neo4j_agent.name}")
print(f"   🧠 Model: {MODEL_NAME}")
print(f"   🛠️  Tools: {len(neo4j_agent.tools)} database tools")
print("   📋 Capabilities:")
print("      - Natural language to Cypher translation")
print("      - Automatic schema discovery")
print("      - Read-only database querying") # Corrected indentation here
print("      - Human-readable result formatting")

🤖 Creating AI agent with Google ADK...
✅ AI agent created successfully!
   🤖 Agent Name: neo4j_database_agent
   🧠 Model: gemini-2.5-flash
   🛠️  Tools: 2 database tools
   📋 Capabilities:
      - Natural language to Cypher translation
      - Automatic schema discovery
      - Read-only database querying
      - Human-readable result formatting


## Step 2.5: Test Agent Locally

**What this does**: Tests your AI agent locally before deploying to the cloud. This helps catch any issues early.

**Test process**:
1. Wraps agent in ADK App for testing
2. Sends a simple query to discover schema
3. Shows real-time interaction steps
4. Verifies database connectivity

**What to expect**: You'll see the agent's "thinking" process as it discovers your database structure and responds to your question.

In [12]:
print("🧪 Testing agent locally...")
print("   This verifies your agent works before cloud deployment.")

# Suppress expected warnings from function calls during testing
import warnings
import logging
warnings.filterwarnings("ignore", message="Warning: there are non-text parts in the response")
logging.getLogger("google_genai.types").setLevel(logging.ERROR)

try:
    # Wrap the agent in an ADK App for testing
    print("\n📦 Wrapping agent in ADK App...")
    test_app = reasoning_engines.AdkApp(agent=neo4j_agent, enable_tracing=True)
    print("✅ Agent wrapped successfully.")

    # Test with a simple query
    test_question = "List all subcriptions with their Mrr sort by product with grand total"
    print(f"\n❓ Test Question: '{test_question}'")
    print("\n🔄 Agent Response (streaming):")
    print("-" * 50)

    # Stream the response to see real-time interaction
    response_parts = []
    for event in test_app.stream_query(
        user_id="local_test_user",
        message=test_question,
    ):
        # Check if this is a text response (final answer)
        if 'content' in event and 'parts' in event['content']:
            for part in event['content']['parts']:
                if 'text' in part:
                    text_content = part['text']
                    print(text_content)
                    response_parts.append(text_content)
                elif 'function_call' in part:
                    function_name = part['function_call']['name']
                    print(f"🔧 Agent is calling: {function_name}")

    print("-" * 50)

    if response_parts:
        print("✅ Local test successful!")
        print("   ✓ Agent connected to database")
        print("   ✓ Schema discovery working")
        print("   ✓ Response generation working")
        print("\n🚀 Ready for deployment!")
    else:
        print("⚠️  Local test completed but no text response received.")
        print("   This might be normal - check the function calls above.")

except Exception as e:
    print(f"❌ Local test failed: {e}")
    print("\n🔧 Troubleshooting:")
    print("   1. Check your Neo4j credentials are correct")
    print("   2. Verify your Neo4j database is accessible")
    print("   3. Ensure your Neo4j instance is running")
    print("   4. Check network connectivity to Neo4j")
    raise

🧪 Testing agent locally...
   This verifies your agent works before cloud deployment.

📦 Wrapping agent in ADK App...
✅ Agent wrapped successfully.

❓ Test Question: 'List all subcriptions with their Mrr sort by product with grand total'

🔄 Agent Response (streaming):
--------------------------------------------------
🔧 Agent is calling: get_neo4j_schema
🔧 Agent is calling: execute_cypher_query
🔧 Agent is calling: execute_cypher_query
Here are the subscriptions with their MRR, sorted by product:

| Product           | MRR     |
|-------------------|---------|
| Basic Plan        | 99.99   |
| Enterprise Plan   | 999.99  |
| Premium Plan      | 299.99  |

🔧 Agent is calling: execute_cypher_query
The grand total MRR for all subscriptions is 1399.97.
--------------------------------------------------
✅ Local test successful!
   ✓ Agent connected to database
   ✓ Schema discovery working
   ✓ Response generation working

🚀 Ready for deployment!


# 🎯 Phase 3: Deployment Preparation

Now that your agent works locally, let's prepare it for cloud deployment.

## Step 3.1: Prepare Deployment Configuration

**What this does**: Sets up the configuration needed for cloud deployment, including environment variables and deployment settings.

**Deployment requirements**:
- Environment variables for database connection
- Python package requirements
- Resource specifications
- Security configurations

In [13]:
print("⚙️  Preparing deployment configuration...")

# Environment variables that will be set in the cloud deployment
deployment_env_vars = {
    "NEO4J_URI": NEO4J_URI,
    "NEO4J_USERNAME": NEO4J_USERNAME,
    "NEO4J_PASSWORD": NEO4J_PASSWORD,
    "NEO4J_DATABASE": NEO4J_DATABASE,
}

# Python packages required for the deployed agent
deployment_requirements = [
    "google-cloud-aiplatform[adk,agent_engines]>=1.56.0",  # Google AI Platform with ADK
    "neo4j>=5.15.0",                                        # Neo4j Python driver
]

# Agent description for the cloud console
deployment_description = (
    f"Intelligent Neo4j database assistant created from template. "
    f"Converts natural language questions into Cypher queries and provides "
    f"human-readable results. Connected to: {NEO4J_URI.split('@')[1] if '@' in NEO4J_URI else NEO4J_URI}"
)

print("✅ Deployment configuration prepared!")
print(f"   🔐 Environment variables: {len(deployment_env_vars)} configured")
print(f"   📦 Python requirements: {len(deployment_requirements)} packages")
print(f"   📝 Description: Ready")
print(f"   🏷️  Display name: {AGENT_DISPLAY_NAME}")

# Validate deployment readiness
print("\n🔍 Deployment readiness check:")
checks = [
    ("Google Cloud project", PROJECT_ID and "your-" not in PROJECT_ID),
    ("Staging bucket", STAGING_BUCKET and "your-" not in STAGING_BUCKET),
    ("Neo4j URI", NEO4J_URI and "your-" not in NEO4J_URI),
    ("Neo4j password", NEO4J_PASSWORD and "your-" not in NEO4J_PASSWORD),
    ("Agent name", AGENT_DISPLAY_NAME and "my-" not in AGENT_DISPLAY_NAME),
]

all_ready = True
for check_name, check_result in checks:
    status = "✅" if check_result else "❌"
    print(f"   {status} {check_name}")
    if not check_result:
        all_ready = False

if all_ready:
    print("\n🎉 All deployment checks passed! Ready to deploy.")
else:
    print("\n⚠️  Some deployment checks failed. Please fix the issues above before deploying.")

⚙️  Preparing deployment configuration...
✅ Deployment configuration prepared!
   🔐 Environment variables: 4 configured
   📦 Python requirements: 2 packages
   📝 Description: Ready
   🏷️  Display name: agent_neo4j_v5

🔍 Deployment readiness check:
   ✅ Google Cloud project
   ✅ Staging bucket
   ✅ Neo4j URI
   ✅ Neo4j password
   ✅ Agent name

🎉 All deployment checks passed! Ready to deploy.


# ☁️ Phase 4: Cloud Deployment

Time to deploy your agent to Google Cloud so others can use it!

## Step 4.1: Deploy to Vertex AI Agent Engine

**What this does**: Deploys your agent to Google Cloud's Agent Engine, making it available as a cloud service.

**Deployment process**:
1. Wraps your agent in a deployable ADK App
2. Runs a final local test
3. Uploads code and dependencies to Google Cloud
4. Creates the agent service
5. Provides access URLs

**Time**: This can take 5-15 minutes. Please be patient!

**What you'll get**: A cloud-hosted agent that your team can access via API or Google Cloud Console.

In [14]:
print("🚀 Starting cloud deployment...")
print(f"   📋 Agent: {AGENT_DISPLAY_NAME}")
print(f"   🌍 Region: {LOCATION}")
print(f"   ⏱️  Estimated time: 5-15 minutes")
print("   ☕ Perfect time for a coffee break!")

def deploy_neo4j_agent():
    """Deploy the Neo4j agent to Vertex AI Agent Engine."""
    try:
        print("\n📦 Step 1: Wrapping agent in deployable ADK App...")
        deployable_app = reasoning_engines.AdkApp(agent=neo4j_agent, enable_tracing=True)
        print("✅ Agent wrapped successfully.")

        print("\n🧪 Step 2: Running final local test...")
        test_events = list(deployable_app.stream_query(
            user_id="deployment_test_user",
            message="Test connection to database",
        ))
        print(f"✅ Final test completed ({len(test_events)} events).")

        print("\n☁️ Step 3: Deploying to Vertex AI Agent Engine...")
        print("   📤 Uploading code and dependencies...")
        print("   🔧 Creating agent service...")
        print("   ⚙️  Configuring environment...")

        # Deploy to Agent Engine
        remote_agent = agent_engines.create(
            deployable_app,
            display_name=AGENT_DISPLAY_NAME,
            description=deployment_description,
            requirements=deployment_requirements,
            env_vars=deployment_env_vars
        )

        print("\n🎉 DEPLOYMENT SUCCESSFUL! 🎉")
        print("\n📋 Deployment Details:")
        print(f"   🤖 Agent Name: {AGENT_DISPLAY_NAME}")
        print(f"   🆔 Resource ID: {remote_agent.resource_name}")
        print(f"   🌍 Region: {LOCATION}")
        print(f"   📊 Model: {MODEL_NAME}")

        print("\n🔗 Access Your Agent:")
        print(f"   📱 Google Cloud Console: https://console.cloud.google.com/vertex-ai?project={PROJECT_ID}")
        print("   🔍 Navigate to: Vertex AI → Agent Builder → Your Agent")

        print("\n✨ Your agent is now live and ready to use!")

        return remote_agent

    except Exception as e:
        print(f"\n❌ DEPLOYMENT FAILED")
        print(f"   Error: {str(e)}")
        print("\n🔧 Troubleshooting Steps:")
        print("   1. Verify the Vertex AI API is enabled in your project")
        print("   2. Check Agent Builder API is enabled")
        print("   3. Ensure your service account has necessary permissions")
        print("   4. Verify your staging bucket is accessible")
        print("   5. Check quota limits in your project")
        print("\n📖 For more help, check the Google Cloud logs:")
        print(f"   https://console.cloud.google.com/logs/query?project={PROJECT_ID}")
        raise

# Run the deployment
deployed_agent = deploy_neo4j_agent()

🚀 Starting cloud deployment...
   📋 Agent: agent_neo4j_v5
   🌍 Region: europe-west4
   ⏱️  Estimated time: 5-15 minutes
   ☕ Perfect time for a coffee break!

📦 Step 1: Wrapping agent in deployable ADK App...
✅ Agent wrapped successfully.

🧪 Step 2: Running final local test...


INFO:vertexai.agent_engines:Identified the following requirements: {'pydantic': '2.11.7', 'cloudpickle': '3.1.1', 'google-cloud-aiplatform': '1.98.0'}
INFO:vertexai.agent_engines:The following requirements are appended: {'pydantic==2.11.7', 'cloudpickle==3.1.1'}
INFO:vertexai.agent_engines:The final list of requirements: ['google-cloud-aiplatform[adk,agent_engines]>=1.56.0', 'neo4j>=5.15.0', 'pydantic==2.11.7', 'cloudpickle==3.1.1']


✅ Final test completed (3 events).

☁️ Step 3: Deploying to Vertex AI Agent Engine...
   📤 Uploading code and dependencies...
   🔧 Creating agent service...
   ⚙️  Configuring environment...


INFO:vertexai.agent_engines:Using bucket vertex_ai_agent_engine_test_disco
INFO:vertexai.agent_engines:Wrote to gs://vertex_ai_agent_engine_test_disco/agent_engine/agent_engine.pkl
INFO:vertexai.agent_engines:Writing to gs://vertex_ai_agent_engine_test_disco/agent_engine/requirements.txt
INFO:vertexai.agent_engines:Creating in-memory tarfile of extra_packages
INFO:vertexai.agent_engines:Writing to gs://vertex_ai_agent_engine_test_disco/agent_engine/dependencies.tar.gz
INFO:vertexai.agent_engines:Creating AgentEngine
INFO:vertexai.agent_engines:Create AgentEngine backing LRO: projects/881765721010/locations/europe-west4/reasoningEngines/7024208043814420480/operations/4445648904955887616
INFO:vertexai.agent_engines:View progress and logs at https://console.cloud.google.com/logs/query?project=881765721010
INFO:vertexai.agent_engines:AgentEngine created. Resource name: projects/881765721010/locations/europe-west4/reasoningEngines/7024208043814420480
INFO:vertexai.agent_engines:To use this 


🎉 DEPLOYMENT SUCCESSFUL! 🎉

📋 Deployment Details:
   🤖 Agent Name: agent_neo4j_v5
   🆔 Resource ID: projects/881765721010/locations/europe-west4/reasoningEngines/7024208043814420480
   🌍 Region: europe-west4
   📊 Model: gemini-2.5-flash

🔗 Access Your Agent:
   📱 Google Cloud Console: https://console.cloud.google.com/vertex-ai?project=881765721010
   🔍 Navigate to: Vertex AI → Agent Builder → Your Agent

✨ Your agent is now live and ready to use!


## Step 4.2: Test Deployed Agent

**What this does**: Tests your deployed agent in the cloud to ensure it works correctly.

**Test process**:
1. Connects to your deployed agent
2. Sends a test query
3. Streams the real-time response
4. Verifies cloud functionality

**Success criteria**: Agent responds with database information, showing it can connect to both Google Cloud and your Neo4j database.

In [15]:
print("🧪 Testing deployed agent...")

# Get the deployed agent resource name
if 'deployed_agent' in locals() and deployed_agent:
    agent_resource_name = deployed_agent.resource_name
    print(f"   🆔 Agent Resource: {agent_resource_name}")
else:
    print("❌ No deployed agent found. Please run the deployment step first.")
    print("   If you have a previously deployed agent, enter its resource name below:")
    # You can manually set the resource name here if needed
    # agent_resource_name = "projects/YOUR_PROJECT/locations/YOUR_REGION/reasoningEngines/YOUR_ID"
    raise SystemExit("Deployed agent not found.")

try:
    print("\n🔗 Connecting to deployed agent...")
    remote_agent = agent_engines.get(agent_resource_name)
    print("✅ Connected successfully!")

    # Test with a comprehensive query
    test_questions = [
        "Show me a summary of what data is in this database",
        "List all the types of nodes in the database",
        "Give me a sample of data from the most interesting node type"
    ]

    for i, question in enumerate(test_questions, 1):
        print(f"\n📝 Test {i}: {question}")
        print("-" * 60)

        try:
            # Stream the response
            response_received = False
            for event in remote_agent.stream_query(
                user_id=f"test_user_{i}",
                message=question,
            ):
                # Check for text responses
                if ('content' in event and
                    'parts' in event['content'] and
                    event['content']['parts']):

                    for part in event['content']['parts']:
                        if 'text' in part:
                            print(part['text'])
                            response_received = True
                        elif 'function_call' in part:
                            function_name = part['function_call']['name']
                            print(f"🔧 Calling: {function_name}")

            if response_received:
                print(f"\n✅ Test {i} completed successfully!")
            else:
                print(f"\n⚠️  Test {i} completed but no text response received.")

        except Exception as e:
            print(f"\n❌ Test {i} failed: {e}")

    print("\n" + "=" * 60)
    print("🎉 DEPLOYMENT AND TESTING COMPLETE! 🎉")
    print("\n✅ Your Neo4j AI Agent is now live and functional!")

    print("\n📋 Summary:")
    print(f"   🤖 Agent Name: {AGENT_DISPLAY_NAME}")
    print(f"   🆔 Resource ID: {agent_resource_name}")
    print(f"   🗄️  Database: Connected to Neo4j")
    print(f"   🧠 Model: {MODEL_NAME}")
    print(f"   🌍 Region: {LOCATION}")

    print("\n🚀 Next Steps:")
    print("   1. Share the Google Cloud Console link with your team")
    print("   2. Test with your own questions via the console")
    print("   3. Integrate via API if needed")
    print("   4. Monitor usage and performance")

    print("\n🔗 Quick Access:")
    print(f"   📱 Console: https://console.cloud.google.com/vertex-ai?project={PROJECT_ID}")
    print("   📖 Documentation: https://cloud.google.com/vertex-ai/docs")

except Exception as e:
    print(f"\n❌ Testing failed: {e}")
    print("\n🔧 Troubleshooting:")
    print("   1. Wait a few minutes for deployment to fully complete")
    print("   2. Check the Google Cloud Console for any errors")
    print("   3. Verify your Neo4j database is still accessible")
    print("   4. Check agent logs in Google Cloud Logging")

🧪 Testing deployed agent...
   🆔 Agent Resource: projects/881765721010/locations/europe-west4/reasoningEngines/7024208043814420480

🔗 Connecting to deployed agent...
✅ Connected successfully!

📝 Test 1: Show me a summary of what data is in this database
------------------------------------------------------------
🔧 Calling: get_neo4j_schema
This Neo4j database contains the following types of data:

**Nodes (Entities):**
*   **SubscriptionEvent:** Represents events related to subscriptions.
*   **Payment:** Stores payment information.
*   **Invoice:** Represents invoices issued.
*   **User:** Contains information about users.
*   **Customer:** Details about customers.
*   **CreditNote:** Represents credit notes.
*   **Service:** Information about services offered.
*   **Subscription:** Details about customer subscriptions.
*   **Case:** Represents customer support cases.

**Relationships:**
*   A **Customer** `HAS_INVOICE` an **Invoice**.
*   A **CreditNote** is `ISSUED_FOR_CUSTOMER` a 

# 🚀 Phase 5: Register Agent in Agentspace
What this does: This phase takes your deployed agent and registers it with
Agentspace, making it discoverable and usable within your organization's
agent gallery.

 Prerequisites:
   1. The agent must be successfully deployed in Phase 4.
   2. The 'agent_registration_tool' files you provided, especially
      'as_agent_registry_service.py', must be uploaded to your notebook
      environment so they can be imported.


### --- Step 5.1: Install Libraries ---


In [16]:
import os
import json
import importlib

print("🚀 Starting Phase 5: Register Agent in Agentspace")

🚀 Starting Phase 5: Register Agent in Agentspace


### --- Step 5.2: Import Registration Service ---
We will dynamically import the function from the script you provided.



In [17]:
print("\n⚙️  Configuring Agentspace registration parameters...")

# This should be the 'App ID' for your Agentspace application.
# It tells the registration service where to list your agent.
AGENTSPACE_APP_ID = input("🔵 Enter your Agentspace App ID: ").strip()
while not AGENTSPACE_APP_ID:
    print("   ❌ Agentspace App ID is required!")
    AGENTSPACE_APP_ID = input("🔵 Enter your Agentspace App ID: ").strip()

# The ADK Deployment ID is the unique identifier for your agent in Vertex AI.
# We extract it from the 'deployed_agent' object created in Phase 4.
try:
    if 'deployed_agent' in locals() and deployed_agent:
        # The resource name is like: projects/PROJECT_ID/locations/LOCATION/reasoningEngines/ADK_DEPLOYMENT_ID
        # We need to get the last part.
        adk_deployment_id = deployed_agent.resource_name.split('/')[-1]
        print(f"   - Extracted ADK Deployment ID: {adk_deployment_id}")
    else:
        print("❌ Could not find 'deployed_agent'. Please ensure Phase 4 ran successfully.")
        raise NameError("deployed_agent not defined")
except Exception as e:
    print(f"   ❌ Failed to get ADK Deployment ID: {e}")
    print("      Please ensure the `deployed_agent` variable from Phase 4 is available.")
    raise

# The 'tool_description' is crucial. It's the detailed instruction that
# tells the master agent (LLM) when to route a user's request to THIS agent.
# We will reuse the detailed agent instruction from Phase 2.
tool_description_for_registration = neo4j_agent.instruction
print("   - Using agent instruction from Phase 2 as the tool description.")

# The user-facing description for the agent in the Agentspace UI.
# We will reuse the deployment description from Phase 3.
description_for_registration = deployment_description
print("   - Using deployment description from Phase 3 for UI.")

# Optional: You can add a public URL to an icon for your agent.
icon_uri_for_registration = input("🖼️ Enter Icon URI (optional, press Enter to skip): ").strip()

print("✅ Configuration complete.")


⚙️  Configuring Agentspace registration parameters...
🔵 Enter your Agentspace App ID: innovation-basement_1741551269956
   - Extracted ADK Deployment ID: 7024208043814420480
   - Using agent instruction from Phase 2 as the tool description.
   - Using deployment description from Phase 3 for UI.
🖼️ Enter Icon URI (optional, press Enter to skip): 
✅ Configuration complete.


### --- Step 5.3: Configure Agentspace Parameters ---
Here, we gather all the necessary information for registration.

In [19]:
import importlib
import sys
import os

# Create the service file directly using Python
service_code = """
import subprocess
import json
import logging
import google.auth
import google.auth.transport.requests

# Scopes define the level of access you are requesting.
SCOPES = ['https://www.googleapis.com/auth/cloud-platform']

def get_access_token():
    \"\"\"Gets the access token from the environment.\"\"\"
    try:
        credentials, project_id = google.auth.default(scopes=SCOPES)
        credentials.refresh(google.auth.transport.requests.Request())
        return credentials.token
    except Exception as e:
        print(f"An error occurred in get_access_token: {e}")
        import traceback
        traceback.print_exc()
        return None

# --- Constants for URL construction ---
BASE_API_URL = "https://{location_prefix}discoveryengine.googleapis.com/v1alpha"
DEFAULT_LOCATION = "global"
DEFAULT_COLLECTION = "default_collection"
DEFAULT_ASSISTANT = "default_assistant"

def _check_required_params(params, required):
    missing = [param for param in required if not params.get(param)]
    if missing:
        raise ValueError(f"Missing required parameters: {', '.join(missing)}")

def _build_discovery_engine_url(project_id: str, app_id: str, agent_id: str = None, api_location: str = DEFAULT_LOCATION) -> str:
    \"\"\"Helper function to construct the Discovery Engine API URL for agents.\"\"\"
    location_prefix = ""
    if api_location != "global":
        location_prefix = api_location + "-"
    url = BASE_API_URL.format(location_prefix=location_prefix)
    base_path = f"{url}/projects/{project_id}/locations/{api_location}/collections/{DEFAULT_COLLECTION}/engines/{app_id}/assistants/{DEFAULT_ASSISTANT}/agents"
    if agent_id:
        return f"{base_path}/{agent_id}"
    return base_path

def create_agent(project_id, app_id, display_name, description, tool_description, adk_deployment_id, auth_id, icon_uri=None, re_location="global", api_location="global"):
    \"\"\"Creates a new agent in the Agent Registry.\"\"\"
    _check_required_params(locals(), ["project_id", "app_id", "display_name", "description", "tool_description", "adk_deployment_id"])
    access_token = get_access_token()
    if not access_token:
        logging.error("Failed to obtain access token for create_agent.")
        return {"status_code": 401, "stdout": "", "stderr": "Failed to obtain access token."}

    url = _build_discovery_engine_url(project_id, app_id, api_location=api_location)
    data = {
        "displayName": display_name,
        "description": description,
        "adk_agent_definition": {
            "tool_settings": {"tool_description": tool_description},
            "provisioned_reasoning_engine": {"reasoning_engine": f"projects/{project_id}/locations/{re_location}/reasoningEngines/{adk_deployment_id}"},
            "authorizations": [f"projects/{project_id}/locations/{api_location}/authorizations/{auth_id}"] if auth_id else [],
        }
    }
    if icon_uri:
        data["icon"] = {"uri": icon_uri}

    command = [
        "curl", "-X", "POST", "-H", f"Authorization: Bearer {access_token}",
        "-H", "Content-Type: application/json", "-H", f"X-Goog-User-Project: {project_id}",
        url, "-d", json.dumps(data)
    ]
    result = subprocess.run(command, capture_output=True, text=True)
    response = {"status_code": result.returncode, "stdout": result.stdout, "stderr": result.stderr}
    if result.returncode == 0:
        try:
            response["agent"] = json.loads(result.stdout)
        except json.JSONDecodeError:
            response["error"] = "Could not decode JSON response."
    return response
"""

# Write the file
with open('as_agent_registry_service.py', 'w') as f:
    f.write(service_code)

print("✅ File 'as_agent_registry_service.py' created successfully!")

# Now import it
try:
    # Remove from sys.modules if it exists to force reload
    if 'as_agent_registry_service' in sys.modules:
        del sys.modules['as_agent_registry_service']

    import as_agent_registry_service
    from as_agent_registry_service import create_agent as register_agent_in_agentspace
    print("✅ Successfully imported Agentspace registration service.")
except ImportError as e:
    print(f"❌ ERROR: Failed to import: {e}")
    raise

✅ File 'as_agent_registry_service.py' created successfully!
✅ Successfully imported Agentspace registration service.


### --- Step 5.4: Register the Agent ---
Now we call the registration function with all our prepared parameters.

In [20]:
print("\n🔄 Registering agent with Agentspace...")

try:
    # Call the registration function from your provided script
    registration_result = register_agent_in_agentspace(
        project_id=PROJECT_ID,
        app_id=AGENTSPACE_APP_ID,
        display_name=AGENT_DISPLAY_NAME,
        description=description_for_registration,
        tool_description=tool_description_for_registration,
        adk_deployment_id=adk_deployment_id,
        auth_id=None,  # Set to your OAuth ID if the agent needs user-level permissions
        icon_uri=icon_uri_for_registration if icon_uri_for_registration else None,
        re_location=LOCATION, # The location/region of your deployed agent
        api_location="eu" # API location set to Europe
    )
except Exception as e:
  print(f"Error during agent registration: {e}")


🔄 Registering agent with Agentspace...


### --- Step 5.5: Display Registration Result ---

In [21]:
if registration_result:
    print("\n🎉 AGENT REGISTRATION ATTEMPT COMPLETE! �")

    # First, try to parse the JSON response.
    try:
        response_data = json.loads(registration_result.get("stdout", "{}"))
    except json.JSONDecodeError:
        print("❌ Registration failed. Could not parse response from server.")
        print("   Raw Response:", registration_result.get("stdout"))
        response_data = None

    if response_data and "error" not in response_data:
        # SUCCESS PATH
        print("✅ Agent was successfully registered in Agentspace.")
        print("\n📋 Registration Details:")
        print(json.dumps(response_data, indent=2))

        # Extract the new Agentspace resource name
        agent_resource_name = response_data.get("name")
        print(f"\n✨ Your agent is now available in Agentspace App '{AGENTSPACE_APP_ID}'")
        print(f"   - Agentspace Resource Name: {agent_resource_name}")
    else:
        # FAILURE PATH
        print("❌ Registration failed.")
        if response_data and "error" in response_data:
            print("\n📋 API Error Details:")
            print(json.dumps(response_data['error'], indent=2))
        else:
            print("   - Status Code:", registration_result.get("status_code"))
            print("   - Raw Error Details:", registration_result.get("stderr"))

        print("\n🔧 Troubleshooting:")
        print("   - Verify your Agentspace App ID is correct and exists in the 'eu' location.")
        print("   - Ensure you have the 'Discovery Engine Admin' role or 'agents.manage' IAM permission.")
        print("   - Check that the Discovery Engine API is enabled for your project.")


🎉 AGENT REGISTRATION ATTEMPT COMPLETE! �
✅ Agent was successfully registered in Agentspace.

📋 Registration Details:
{
  "name": "projects/881765721010/locations/eu/collections/default_collection/engines/innovation-basement_1741551269956/assistants/default_assistant/agents/6769061732880912825",
  "displayName": "agent_neo4j_v5",
  "description": "Intelligent Neo4j database assistant created from template. Converts natural language questions into Cypher queries and provides human-readable results. Connected to: neo4j+s://agentspace-c4x9-c86d.endpoints.neo4j.io",
  "createTime": "2025-06-24T19:25:55.228122318Z",
  "adkAgentDefinition": {
    "toolSettings": {
      "toolDescription": "You are an expert Neo4j database assistant. Your role is to help users get insights from their Neo4j database using natural language queries.\n\nWORKFLOW:\n1. ALWAYS start by calling `get_neo4j_schema` to understand the database structure\n2. Analyze the schema to understand nodes, relationships, and prope

# 🎊 Congratulations!

You've successfully created and deployed a Neo4j AI Agent using Google's Agent Development Kit!

## 🌟 What You've Accomplished

✅ **Built an intelligent database assistant** that understands natural language  
✅ **Deployed to Google Cloud** for team access  
✅ **Connected to your Neo4j database** securely  
✅ **Created a reusable template** for future projects  





# Test of deployment and Deletion script



In [ ]:
from vertexai import agent_engines
import logging

print("🚀 Starting Phase 7: Direct Agent Engine Test")

# We use the 'deployed_agent' object from Phase 4.

try:
    if 'deployed_agent' in locals() and deployed_agent:
        agent_resource_name = deployed_agent.resource_name
        print(f"   - Found Agent Engine Resource Name: {agent_resource_name}")
    else:
        # If the notebook was restarted, you can manually paste the resource name here
        # agent_resource_name = "projects/test-disco-cm/locations/europe-west4/reasoningEngines/2905666184584101888"
        print("❌ Could not find 'deployed_agent' object.")
        print("   Please run Phase 4 again, or manually set the 'agent_resource_name'.")
        raise NameError("deployed_agent not found")
except Exception as e:
    print(f"   ❌ An error occurred getting the resource name: {e}")
    raise

In [ ]:
print("\n🔄 Attempting to connect directly to the Agent Engine...")
try:
    # Get a client for the remote agent
    remote_agent_client = agent_engines.get(agent_resource_name)
    print("✅ Successfully connected to the Agent Engine client.")

    test_question = "What is the schema of the database?"
    print(f"\n❓ Sending test question: '{test_question}'")

   # Use the correct 'stream_query' method and loop through the events
    response_received = False
    full_response = ""
    for event in remote_agent_client.stream_query(message=test_question):
        if 'content' in event and 'parts' in event['content']:
            for part in event['content']['parts']:
                if 'text' in part:
                    print(part['text'], end="", flush=True)
                    full_response += part['text']
                    response_received = True
                elif 'function_call' in part:
                    print(f"\n🔧 Agent is calling: {part['function_call']['name']}\n")
except Exception as e:
    print(f"\n❌ Error connecting to or querying the Agent Engine: {e}")

if not response_received:
    print("\n\n\n❌ Did not receive a response from the Agent Engine.")
else:
    print("\n\n\n✅ Successfully received a response from the Agent Engine.")

In [ ]:
from vertexai import agent_engines
import logging

print("🚀 Starting Phase 7: Direct Agent Engine Test")

# We use the 'deployed_agent' object from Phase 4.

try:
    if 'deployed_agent' in locals() and deployed_agent:
        agent_resource_name = deployed_agent.resource_name
        print(f"   - Found Agent Engine Resource Name: {agent_resource_name}")
    else:
        # If the notebook was restarted, you can manually paste the resource name here
        # agent_resource_name = "projects/test-disco-cm/locations/europe-west4/reasoningEngines/2905666184584101888"
        print("❌ Could not find 'deployed_agent' object.")
        print("   Please run Phase 4 again, or manually set the 'agent_resource_name'.")
        raise NameError("deployed_agent not found")

    print("\n🔄 Attempting to connect directly to the Agent Engine...")

    # Get a client for the remote agent
    remote_agent_client = agent_engines.get(agent_resource_name)
    print("✅ Successfully connected to the Agent Engine client.")

    test_question = "What is the schema of the database?"
    print(f"\n❓ Sending test question: '{test_question}'")

    # Send the query directly using the correct 'chat' method
    response = remote_agent_client.chat(message=test_question)

    # This code was previously in ipython-input-55-3830080924.py
    print("\n📝 Direct Response from Agent Engine:")
    print(response)
    print("\n✅ Direct test completed successfully!")
    print("   This indicates the deployed agent is working correctly, and the issue lies")
    print("   within the Agentspace service or its connection to the agent.")

except Exception as e:
    # This code was previously in ipython-input-55-3830080924.py
    print("\n❌ Direct test failed.")
    logging.exception("   The following error occurred during the direct test:")
    print("\n🔧 This error means the problem is with the Agent Engine deployment itself,")
    print("   not with Agentspace. The error message above should provide clues.")
    print("   Common causes include networking (firewalls) or incorrect env variables.")

## 🚀 Delete Agent Registration from Agentspace

What this does: This script deletes a registered agent from an Agentspace
application. This action is irreversible.

In [ ]:
import os
import json
import importlib

print("🚀 Starting Phase 6: Delete Agent from Agentspace")

# --- Step 6.1: Create and Import Deletion Service ---
# We will create a new file with a dedicated delete function.

get_ipython().run_cell_magic('writefile', 'as_agent_deletion_service.py',
"""
import subprocess
import json
import logging
import google.auth
import google.auth.transport.requests

# Scopes define the level of access you are requesting.
SCOPES = ['https://www.googleapis.com/auth/cloud-platform']

def get_access_token():
    \"\"\"Gets the access token from the environment.\"\"\"
    try:
        # We only need the credentials object from here.
        credentials, _ = google.auth.default(scopes=SCOPES)
        credentials.refresh(google.auth.transport.requests.Request())
        return credentials.token
    except Exception as e:
        print(f"An error occurred in get_access_token: {e}")
        return None

def delete_agent(agent_resource_name: str, project_id: str):
    \"\"\"Deletes an agent registration from Agentspace using its full resource name.\"\"\"
    if not agent_resource_name:
        raise ValueError("agent_resource_name cannot be empty.")
    if not project_id:
        raise ValueError("project_id cannot be empty.")

    access_token = get_access_token()
    if not access_token:
        logging.error("Failed to obtain access token for delete_agent.")
        return {"status_code": 401, "stdout": "", "stderr": "Failed to obtain access token."}

    # The API endpoint is the resource name itself, prefixed with the base URL.
    # We need to extract the API location (e.g., 'eu') from the name for the URL prefix.
    try:
        parts = agent_resource_name.split('/')
        location = parts[3]
        location_prefix = ""
        if location and location != "global":
            location_prefix = location + "-"

        url = f"https://{location_prefix}discoveryengine.googleapis.com/v1alpha/{agent_resource_name}"
    except IndexError:
        return {"status_code": 400, "stdout": "", "stderr": "Invalid agent_resource_name format."}


    command = [
        "curl", "-X", "DELETE",
        "-H", f"Authorization: Bearer {access_token}",
        "-H", "Content-Type: application/json",
        # Use the explicitly passed project_id
        "-H", f"X-Goog-User-Project: {project_id}",
        url
    ]

    print(f"Executing command to delete: {agent_resource_name}")
    result = subprocess.run(command, capture_output=True, text=True)

    response = {"status_code": result.returncode, "stdout": result.stdout, "stderr": result.stderr}
    return response
"""
)

# Now that the file is created, we can import it.
try:
    import as_agent_deletion_service
    importlib.reload(as_agent_deletion_service)
    from as_agent_deletion_service import delete_agent as delete_agent_from_agentspace
    print("✅ Successfully imported Agentspace deletion service.")
except ImportError:
    print("❌ ERROR: 'as_agent_deletion_service.py' not found.")
    raise

# --- Step 6.2: Get Agent Resource Name to Delete ---
print("\n📝 Please provide the full resource name of the agent you wish to delete.")
print("   Example: projects/your-project/locations/eu/collections/default_collection/engines/your-app/assistants/default_assistant/agents/12345")

agent_to_delete = input("🔴 Agent Resource Name: ").strip()
while not agent_to_delete:
    print("   ❌ Agent Resource Name is required!")
    agent_to_delete = input("🔴 Agent Resource Name: ").strip()

# --- Step 6.3: Confirm and Execute Deletion ---
print(f"\n⚠️  You are about to permanently delete the agent registration:")
print(f"   {agent_to_delete}")
confirmation = input("   Are you sure you want to proceed? (yes/no): ").strip().lower()

if confirmation == 'yes':
    print("\n🔄 Deleting agent registration...")
    try:
        # Check if PROJECT_ID exists from Phase 1
        if 'PROJECT_ID' not in locals():
             raise NameError("PROJECT_ID is not defined. Please run Phase 1 of the notebook.")

        # Pass the PROJECT_ID to the deletion function
        deletion_result = delete_agent_from_agentspace(
            agent_resource_name=agent_to_delete,
            project_id=PROJECT_ID
        )

        if deletion_result.get("status_code") == 0:
            print("✅ Agent registration deleted successfully.")
        else:
            print("❌ Deletion failed.")
            print("   - Status Code:", deletion_result.get("status_code"))
            # Try to parse and print error from stdout if available
            try:
                error_details = json.loads(deletion_result.get("stdout", "{}"))
                print("\n📋 API Error Details:")
                print(json.dumps(error_details.get('error', {}), indent=2))
            except json.JSONDecodeError:
                print("   - Raw Error Details (stderr):", deletion_result.get("stderr"))

    except Exception as e:
        print(f"\n❌ An unexpected error occurred during deletion: {e}")
        import traceback
        traceback.print_exc()
else:
    print("\n🚫 Deletion cancelled.")